In [1]:
]st

Status `/mnt/E4E0A9C0E0A998F6/github/ReinforcementLearningAnIntroduction.jl/notebooks/Project.toml`
  [31c24e10] Distributions v0.22.6
  [91a5bcdd] Plots v0.29.9
  [02c1da58] ReinforcementLearningAnIntroduction v0.2.0 [`..`]
  [e575027e] ReinforcementLearningBase v0.7.1
  [de1b191a] ReinforcementLearningCore v0.3.0
  [2913bbd2] StatsBase v0.32.2
  [f3b207a7] StatsPlots v0.12.0
  [2f01184e] SparseArrays 


In [2]:
using ReinforcementLearningAnIntroduction
using ReinforcementLearningAnIntroduction.TicTacToe
env = TicTacToeEnv()

┌ Info: Precompiling ReinforcementLearningAnIntroduction [02c1da58-b9a1-11e8-0212-f9611b8fe936]
└ @ Base loading.jl:1260
┌ Info: Skipping precompilation since __precompile__(false). Importing ReinforcementLearningAnIntroduction [02c1da58-b9a1-11e8-0212-f9611b8fe936].
└ @ Base loading.jl:1033


___
___
___
isdone = [false], winner = [nothing]


In [3]:
get_current_player(env)

X

In [4]:
observation_space, action_space = get_observation_space(env), get_action_space(env)
nstates, nactions = length(observation_space), length(action_space)

(5478, 10)

If you are curious why there are `5478` states, you may see the discussions [here](https://math.stackexchange.com/questions/485752/tictactoe-state-space-choose-calculation/485852)

In [5]:
observe(env)

(reward = 0.0, terminal = false, state = 4244, legal_actions_mask = Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

Now we'll use the Monte Carlo based method to estimate the value of each state for each player. Think about this, if we have the precise estimation of each state after taking some specific observation according to current observation, then we can just choose the action which leads to the maximum estimation.

So we can create a table for each player first. By default we can set the estimations of all the states to `0.0`. Usually it won't be a problem, but here we can initialize it with a better starting point. For each state, we can check that if the state is a final state or not and set the initial estimation accordingly.

In [6]:
function init_table(role)
    table = zeros(nstates)
    for i in 1:nstates
        s = TicTacToe.ID2STATE[i]
        isdone, winner = TicTacToe.STATES_INFO[s]
        if isdone
            if winner === nothing
                table[i] = 0.5
            elseif winner === role
                table[i] = 1.
            else
                table[i] = 0.
            end
        else
            table[i] = 0.5
        end
    end
    table
end

init_table (generic function with 1 method)

Then we wrap the table in a `TabularApproximator`.

In [7]:
V1 = TabularApproximator(init_table(TicTacToe.offensive));
V2 = TabularApproximator(init_table(TicTacToe.defensive));

And wrap it around a MonteCarloLearner.

In [8]:
learner_1 = MonteCarloLearner(;approximator=V1, α=0.1, kind=EVERY_VISIT)
learner_2 = MonteCarloLearner(;approximator=V2, α=0.1, kind=EVERY_VISIT)

MonteCarloLearner{ReinforcementLearningAnIntroduction.EveryVisit,TabularApproximator{1,Array{Float64,1}},CachedSampleAvg{Float64},ReinforcementLearningAnIntroduction.NoSampling}(TabularApproximator{1,Array{Float64,1}}([0.5, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.5, 0.5, 0.5]), 1.0, 0.1, CachedSampleAvg{Float64}(Dict{Float64,SampleAvg}()))

Then the learner is assemble into a policy.

A policy is a mapping from states to actions. Considering that we already have the estimations of states, a simple policy would be checking the estimation of the following up states and select one action which will result to the best state.

In [9]:
function create_mapping(role)
    (obs, learner) -> begin
        mask = get_legal_actions_mask(obs)
        [
            mask[a] ? learner(StateOverriddenObs(obs=obs, state=TicTacToe.get_next_state_id(get_state(obs), role, a))) : 0.  # a dummy value     
            for a in action_space
        ]
    end
end

create_mapping (generic function with 1 method)

In [10]:
ϵ = 0.01

π_1 = VBasedPolicy(
    learner = learner_1,
    mapping = create_mapping(TicTacToe.offensive),
    explorer = EpsilonGreedyExplorer(ϵ;seed=1),
    )

π_2 = VBasedPolicy(
    learner = learner_2,
    mapping = create_mapping(TicTacToe.defensive),
    explorer = EpsilonGreedyExplorer(ϵ;seed=2),
    );

agent_1 = Agent(
    policy = π_1,
    trajectory = EpisodicCompactSARTSATrajectory(),
    role=TicTacToe.offensive
    );

agent_2 = Agent(
    policy = π_2,
    trajectory = EpisodicCompactSARTSATrajectory(),
    role=TicTacToe.defensive
    );

reset!(env)

agents = (agent_1, agent_2);

In [11]:
run((agent_1, agent_2), env, StopAfterEpisode(1_000_000))  # try adjusting the number of episodes to see the performance difference

Progress: 100%|█████████████████████████████████████████| Time: 0:07:5139m


2-element Array{EmptyHook,1}:
 EmptyHook()
 EmptyHook()

In [12]:
agent_1.policy.explorer.ϵ_stable = 0.0
agent_2.policy.explorer.ϵ_stable = 0.0

0.0

In [13]:
reset!(env)

Now it's your turn to play this game!

In [14]:
function read_action_from_stdin()
    print("Your input:")
    input = parse(Int, readline())
    !in(input, 1:9) && error("invalid input!")
    input
end

function play()
    env = TicTacToeEnv()
    println("""You play first!
    1 4 7
    2 5 8
    3 6 9""")
    while true
        action = read_action_from_stdin()
        env(action)
        println(env)
        obs = observe(env, TicTacToe.offensive)
        if get_terminal(obs)
            if get_reward(obs) == 0.5
                println("Tie!")
            elseif get_reward(obs) == 1.0 
                println("You win!")
            else
                println("Invalid input!")
            end
            break
        end

        env(agent_2(PRE_ACT_STAGE, observe(env)))
        println(env)
        obs = observe(env, TicTacToe.defensive)
        if get_terminal(obs)
            if get_reward(obs) == 0.5
                println("Tie!")
            elseif get_reward(obs) == 1.0 
                println("Your lose!")
            else
                println("You win!")
            end
            break
        end
    end
end

play (generic function with 1 method)

In [15]:
play()

You play first!
1 4 7
2 5 8
3 6 9
Your input:stdin> 5
___
_X_
___
isdone = [false], winner = [nothing]

___
_X_
__O
isdone = [false], winner = [nothing]

Your input:stdin> 2
___
XX_
__O
isdone = [false], winner = [nothing]

___
XXO
__O
isdone = [false], winner = [nothing]

Your input:stdin> 7
__X
XXO
__O
isdone = [false], winner = [nothing]

__X
XXO
O_O
isdone = [false], winner = [nothing]

Your input:stdin> 6
__X
XXO
OXO
isdone = [false], winner = [nothing]

_OX
XXO
OXO
isdone = [false], winner = [nothing]

Your input:stdin> 1
XOX
XXO
OXO
isdone = [true], winner = [nothing]

Tie!
